### Create cert for the local client

In [1]:
import os

TLSDIR = os.path.join(os.getcwd() + "/tls")

In [2]:
%%bash -s "$TLSDIR"
TLSDIR=$1
mkdir -p ${TLSDIR}

# Download the ca keypair.
oc get secret ca-secret -o template='{{index .data "ca.key"}}'|base64 -d > ${TLSDIR}/ca.key
oc get secret ca-secret -o template='{{index .data "ca.crt"}}'|base64 -d > ${TLSDIR}/ca.crt

# Create certificate signing request
openssl req -nodes -newkey rsa:2048 -keyout ${TLSDIR}/server.key -out ${TLSDIR}/server.csr -subj '/CN=local'
cat <<EOF >${TLSDIR}/domain.ext
authorityKeyIdentifier=keyid,issuer
basicConstraints=CA:FALSE
subjectAltName = @alt_names
[alt_names]
DNS.1 = 127.0.0.1
DNS.2 = localhost
DNS.3 = $(hostname)
EOF

# Sign the certificate with the same ca keypair.
openssl x509 -req -CA ${TLSDIR}/ca.crt -CAkey ${TLSDIR}/ca.key -in ${TLSDIR}/server.csr -out ${TLSDIR}/server.crt -days 365 -CAcreateserial -extfile ${TLSDIR}/domain.ext

# Verify your local device hostname is in the Subject Alternative Name below
openssl x509 -text -noout -in ${TLSDIR}/server.crt |grep "Subject Alternative Name" -A1

Generating a 2048 bit RSA private key
..............................................................+++
.............................+++
writing new private key to '/Users/tedchang/sandbox/try-ray/codeflare-sdk/demo-notebooks/interactive/tls/server.key'
-----
Signature ok
subject=/CN=local
Getting CA Private Key


            X509v3 Subject Alternative Name: 
                DNS:127.0.0.1, DNS:localhost, DNS:tedchangs-MBP.T-mobile.com


### Connect via the rayclient route

In [3]:
import openshift as oc

routes=oc.selector("route").objects()
rayclient_url=""
for r in routes:
    if r.name() == "rayclient":
        rayclient_url=r.model.spec.host
print(rayclient_url)

rayclient-default.apps.ted412.cp.fyre.ibm.com


In [4]:
import os

os.environ["RAY_USE_TLS"] = "1"
os.environ["RAY_TLS_SERVER_CERT"] = os.path.join(TLSDIR, "server.crt")
os.environ["RAY_TLS_SERVER_KEY"] = os.path.join(TLSDIR, "server.key")
os.environ["RAY_TLS_CA_CERT"] = os.path.join(TLSDIR, "ca.crt")

import ray

ray.shutdown()
ray.init(address="ray://"+rayclient_url)

Python version:,3.8.13
Ray version:,2.1.0
Dashboard:,http://10.254.20.17:8265


In [5]:
import math
import ray

@ray.remote
def heavy_calculation_part(num_iterations):
    result = 0.0
    for i in range(num_iterations):
        for j in range(num_iterations):
            for k in range(num_iterations):
                result += math.sin(i) * math.cos(j) * math.tan(k)
    return result
@ray.remote
def heavy_calculation(num_iterations):
    results = ray.get([heavy_calculation_part.remote(num_iterations//30) for _ in range(30)])
    return sum(results)


In [6]:
ref = heavy_calculation.remote(3000)

In [7]:
ray.get(ref)

1789.4644387076714

In [149]:
ray.cancel(ref)